In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/project6/lit-master/lit-master')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
#from matplotlib.mlab import griddata
from scipy.interpolate import griddata
import tensorflow as tf
from neural_network import *
from ensembling_methods import *

## Define plotting helpers

In [ ]:
def contour(f, lines=None, linewidths=6, alpha=0.9, linestyles='solid', **kw):
  set_lims()
  grid = np.linspace(-10, 10, 150)
  xg, yg = np.meshgrid(grid, grid)
  zg = griddata(X[:,0], X[:,1], f, grid, grid, interp='linear')
  if lines is not None:
    plt.contour(xg, yg, zg, lines, linewidths=linewidths, alpha=alpha, linestyles=linestyles, **kw, vmin=-1, vmax=1)
  else:
    vlim = np.abs(f).max() * 0.95
    plt.contourf(xg, yg, zg, 300, vmin=-vlim, vmax=vlim, **kw)
      
def scat(x,y,per=2, **kw):
  set_lims()
  y1 = np.argwhere(y > 0)[:,0]; np.random.shuffle(y1)
  y2 = np.argwhere(y <= 0)[:, 0]; np.random.shuffle(y2)
  plt.scatter(*x[y1][::per].T, marker='.', c='red', edgecolor=(0,0,0,0.25), s=100, **kw)
  plt.scatter(*x[y2][::per].T, marker='.', c='blue', edgecolor=(0,0,0,0.25), s=100, **kw)
  
def set_lims():
  plt.xticks([]); plt.yticks([])
  plt.xlim(-10, 10); plt.ylim(-10, 10)

## Visualize datasets

In [ ]:
def get_logits(m1,m2):
  res = []
  with tf.compat.v1.Session() as sess:
    m1.init(sess)
    m2.init(sess)
    res.append(m1.batch_eval(sess, m1.binary_logits,X))
    res.append(m2.batch_eval(sess, m2.binary_logits,X))
  tf.compat.v1.reset_default_graph()
  return res

def train_lit(X_,y_,overlap_penalty):
  tf.compat.v1.reset_default_graph()
  return get_logits(*train_diverse_models(Net,2,X_,y_,lambda_overlap=overlap_penalty,num_epochs=200,learning_rate=0.0001))
  
def train_rrs(X_,y_):
  tf.compat.v1.reset_default_graph()
  return get_logits(*train_restart_models(Net,2,X_,y_,num_epochs=200))

def train_ncl(X_,y_,overlap_penalty):
  tf.compat.v1.reset_default_graph()
  return get_logits(*train_neg_corr_models(Net,2,X_,y_,lambda_overlap=overlap_penalty,num_epochs=200,learning_rate=0.0001))

In [ ]:
# We'll use a simple 2x256x2 fully connected network
class Net(NeuralNetwork):
  @property
  def x_shape(self): return [None, 24]
  @property
  def y_shape(self): return [None, 2]
  def rebuild_model(self, X, **_):
    # We use softplus here because ReLU networks appear to be unable to get high train accuracy
    # on the 2xy vs. x^2-y^2 dataset, even trained normally (not in an ensemble)
    L0 = X
    L1 = tf.compat.v1.layers.dense(L0, 256, name=self.name+'/L1', activation=tf.nn.softplus)
    L2 = tf.compat.v1.layers.dense(L1,   2, name=self.name+'/L2', activation=None)
    return [L1, L2]

In [ ]:
def train_ncl_with_highest_accurate_penalty(X, y, steps=10, verbose=False):
  lower_bound = 1.0
  upper_bound = 2.0
  best_logits = None
  for step in range(steps):
    lmb = (lower_bound + upper_bound) / 2.0
    m1, m2 = train_neg_corr_models(Net, 2, X, y, lambda_overlap=lmb, num_epochs=100, learning_rate=0.0001)
    with tf.compat.v1.Session() as sess:
      m1.init(sess)
      m2.init(sess)
      log1 = m1.batch_eval(sess, m1.binary_logits, X)
      log2 = m2.batch_eval(sess, m2.binary_logits, X)
    acc1 = np.mean((log1>0)==y)
    acc2 = np.mean((log2>0)==y)
    if verbose:
      print('NCL λ={}: acc1={:.2f}, acc2={:.2f}'.format(lmb, acc1, acc2))
    if acc1 > 0.9 and acc2 > 0.9:
      best_logits = get_logits(m1, m2)
      lower_bound = lmb
    elif acc1 < 0.501 and acc2 < 0.501:
      pass # we got unlucky with training, try again
    else:
      upper_bound = lmb
  #assert(best_logits is not None)
  return lmb, best_logits

In [ ]:
from glob import glob

for files in glob('/content/drive/MyDrive/Colab Notebooks/project6/lit-master/lit-master/processed_8/processed_8/*'):
    # print(files)
    pc = []
    sd_pc = []
    for items in glob("{}/*.*".format(files)):
        print(items)
        # print(len(items))

        data = np.load(items, allow_pickle=True)
        input_list = data.tolist()  # How to get x_train

        #print(input_list)
        x_train = input_list['FrameStack'][0]
        X = x_train
        x_test = input_list['FrameStack'][1]
        y_train = input_list['FrameStack'][2]
        y_test = input_list['FrameStack'][3]
        x_val = input_list['FrameStack'][4]
        y_val = input_list['FrameStack'][5]
        y_shape = max(y_train)+1


        # We'll use a simple 2x256x2 fully connected network
        class Net(NeuralNetwork):
          @property
          def x_shape(self): return [None, x_train.shape[1]]
          @property
          def y_shape(self): return [None, y_shape]
          def rebuild_model(self, X, **_):
            # We use softplus here because ReLU networks appear to be unable to get high train accuracy
            # on the 2xy vs. x^2-y^2 dataset, even trained normally (not in an ensemble)
            L0 = X
            L1 = tf.compat.v1.layers.dense(L0, 256, name=self.name+'/L1', activation=tf.nn.softplus)
            L2 = tf.compat.v1.layers.dense(L1,  y_shape, name=self.name+'/L2', activation=None)
            return [L1, L2]
        results1 = {}
        best_ncl_lambda, best_ncl_logits = train_ncl_with_highest_accurate_penalty(x_train, y_train, verbose=True)
        results1['ncl-best'] = best_ncl_logits
        results1['ncl-{}'.format(0.1)] = train_ncl(x_train, y_train, 0.1)
        results1['restarts'] = train_rrs(x_train, y_train)
        for lmb in [0.0001,0.1]:
          results1['lit-{}'.format(lmb)] = train_lit(x_train, y_train, lmb)